# 2.4 NORMALIZACIÓN DEL DATASET RETAIL

En este notebook se realiza la etapa de **Normalización** del dataset del proyecto:

**`retail_sales_transformado.csv`**

Esta etapa es importante porque:

- Homogeniza las escalas de las variables numéricas.
- Evita que atributos con valores grandes dominen a los modelos de Machine Learning.
- Mejora el desempeño de algoritmos como:
  - Regresión Logística
  - K-Means
  - Modelos basados en distancias
  - Modelos que usan gradiente

Se aplican dos métodos:

1. **Min-Max Scaler** → Normalización (0 a 1)  
2. **Standard Scaler** → Estandarización (media = 0, desviación estándar = 1)

Al final se genera:

- **`retail_sales_escalado_minmax.csv`**  
- **`retail_sales_escalado_standard.csv`**

Ambos serán usados más adelante en la etapa de Modelado.


In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Cargar dataset transformado (generado en 2.3)
df = pd.read_csv("retail_sales_transformado.csv")

df.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total,anio,mes,dia,nombre_mes,rango_edad,monto_calculado,diferencia_monto,nivel_compra,es_compra_alta
0,1,2023-11-24,CUST001,MALE,34,Beauty,3,50,150,2023,11,24,November,Adulto,150,0,Baja,False
1,2,2023-02-27,CUST002,FEMALE,26,Clothing,2,500,1000,2023,2,27,February,Adulto,1000,0,Alta,True
2,3,2023-01-13,CUST003,MALE,50,Electronics,1,30,30,2023,1,13,January,Mayor,30,0,Baja,False
3,4,2023-05-21,CUST004,MALE,37,Clothing,1,500,500,2023,5,21,May,Adulto,500,0,Media,False
4,5,2023-05-06,CUST005,MALE,30,Beauty,2,50,100,2023,5,6,May,Adulto,100,0,Baja,False


## 1. Selección de columnas numéricas a normalizar

En esta etapa se normalizan únicamente las columnas numéricas que participan en los modelos:

- `edad`
- `cantidad`
- `precio_unitario`
- `monto_total`

Columnas como `anio`, `mes` o `dia` NO se normalizan porque son categóricas o temporales.

Primero verifico los tipos de datos.


In [2]:
df.dtypes


id_transaccion         int64
fecha                 object
id_cliente            object
genero                object
edad                   int64
categoria_producto    object
cantidad               int64
precio_unitario        int64
monto_total            int64
anio                   int64
mes                    int64
dia                    int64
nombre_mes            object
rango_edad            object
monto_calculado        int64
diferencia_monto       int64
nivel_compra          object
es_compra_alta          bool
dtype: object

## 2. Crear un subconjunto con las columnas numéricas a normalizar

Esto facilita aplicar los escaladores sin afectar otras columnas.


In [3]:
columnas_numericas = ['edad', 'cantidad', 'precio_unitario', 'monto_total']

df_numerico = df[columnas_numericas].copy()

df_numerico.head()


,edad,cantidad,precio_unitario,monto_total
0,34,3,50,150
1,26,2,500,1000
2,50,1,30,30
3,37,1,500,500
4,30,2,50,100


## 3. Aplicar normalización Min-Max (0 a 1)

Esta técnica ajusta cada valor usando:

\[
X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}
\]

Beneficios:
- Se mantiene la forma de la distribución original.
- Muy útil para algoritmos basados en distancias (ej. K-Means).

Aplicamos MinMaxScaler de sklearn.


In [4]:
scaler_minmax = MinMaxScaler()

df_minmax = scaler_minmax.fit_transform(df_numerico)

df_minmax = pd.DataFrame(df_minmax, columns=[col + '_minmax' for col in columnas_numericas])

df_minmax.head()


,edad_minmax,cantidad_minmax,precio_unitario_minmax,monto_total_minmax
0,0.347826,0.666667,0.052632,0.063291
1,0.173913,0.333333,1.000000,0.493671
2,0.695652,0.000000,0.010526,0.002532
3,0.413043,0.000000,1.000000,0.240506
4,0.260870,0.333333,0.052632,0.037975


## 4. Aplicar Standard Scaler (media = 0, desviación estándar = 1)

Estandarización:

\[
X_{std} = \frac{X - \mu}{\sigma}
\]

Beneficios:
- Útil para métodos basados en gradiente.
- Hace que todos los atributos contribuyan por igual.


In [5]:
scaler_std = StandardScaler()

df_standard = scaler_std.fit_transform(df_numerico)

df_standard = pd.DataFrame(df_standard, columns=[col + '_std' for col in columnas_numericas])

df_standard.head()


,edad_std,cantidad_std,precio_unitario_std,monto_total_std
0,-0.540565,0.429265,-0.685123,-0.546704
1,-1.125592,-0.453996,1.688464,0.971919
2,0.629489,-1.337258,-0.790615,-0.761098
3,-0.321180,-1.337258,1.688464,0.078611
4,-0.833078,-0.453996,-0.685123,-0.636035


## 5. Unir los resultados de normalización con el dataframe original

Para conservar TODAS las columnas:
- Conservo el DataFrame original
- Agrego columnas MinMax normalizadas
- Agrego columnas Standard normalizadas


In [6]:
df_normalizado = pd.concat([df, df_minmax, df_standard], axis=1)

df_normalizado.head()


,id_transaccion,fecha,id_cliente,genero,edad,categoria_producto,cantidad,precio_unitario,monto_total,anio,...,nivel_compra,es_compra_alta,edad_minmax,cantidad_minmax,precio_unitario_minmax,monto_total_minmax,edad_std,cantidad_std,precio_unitario_std,monto_total_std
0,1,2023-11-24,CUST001,MALE,34,Beauty,3,50,150,2023,...,Baja,False,0.347826,0.666667,0.052632,0.063291,-0.540565,0.429265,-0.685123,-0.546704
1,2,2023-02-27,CUST002,FEMALE,26,Clothing,2,500,1000,2023,...,Alta,True,0.173913,0.333333,1.000000,0.493671,-1.125592,-0.453996,1.688464,0.971919
2,3,2023-01-13,CUST003,MALE,50,Electronics,1,30,30,2023,...,Baja,False,0.695652,0.000000,0.010526,0.002532,0.629489,-1.337258,-0.790615,-0.761098
3,4,2023-05-21,CUST004,MALE,37,Clothing,1,500,500,2023,...,Media,False,0.413043,0.000000,1.000000,0.240506,-0.321180,-1.337258,1.688464,0.078611
4,5,2023-05-06,CUST005,MALE,30,Beauty,2,50,100,2023,...,Baja,False,0.260870,0.333333,0.052632,0.037975,-0.833078,-0.453996,-0.685123,-0.636035


## 6. Exportar los datasets normalizados

Se generan dos archivos:

- `retail_sales_escalado_minmax.csv`
- `retail_sales_escalado_standard.csv`

Ambos serán usados en:

- Modelos de Regresión Lineal
- Regresión Logística
- K-Means


In [7]:
df_minmax.to_csv("retail_sales_escalado_minmax.csv", index=False)
df_standard.to_csv("retail_sales_escalado_standard.csv", index=False)

print("✅ Archivos de normalización generados correctamente:")
print(" - retail_sales_escalado_minmax.csv")
print(" - retail_sales_escalado_standard.csv")


✅ Archivos de normalización generados correctamente:
 - retail_sales_escalado_minmax.csv
 - retail_sales_escalado_standard.csv


#  Conclusión de la etapa 2.4 — Normalización

En esta fase se logró escalar correctamente las variables numéricas del dataset Retail usando dos métodos fundamentales:

###  Min-Max Scaler (0–1)
- Ideal para modelos basados en distancia (K-Means).
- Mantiene la forma original de los datos.
- Normaliza los atributos al mismo rango.

###  Standard Scaler (Z-score)
- Centra los datos en media 0 y varianza 1.
- Muy útil para algoritmos como Regresión Logística y modelos lineales.

###  Archivos generados:
- `retail_sales_escalado_minmax.csv`
- `retail_sales_escalado_standard.csv`

Estos archivos serán utilizados en las siguientes etapas:
- 2.5 Análisis Exploratorio (EDA)
- 2.6 Modelo de Regresión Lineal
- 2.7 Regresión Logística
- 2.8 Validación de modelos

Con esta etapa completada, el dataset ya está completamente preparado para entrar a la fase de análisis y modelado.
